In [2]:
import yfinance as yf
import pandas as pd
import plotly.express as px

'''
análise gráfica de força relativa com momentum de um ativo da B3 (Bolsa brasileira), comparando-o com o índice IBOVESPA
'''

# Definindo o período de análise
start_date = "2023-01-01"
end_date = "2024-01-01"

# Solicitando o ticker de referência
ticker = str(input('\nDigite o ticker Referência: ') + ".SA").upper()

# Baixando dados do ativo e do IBOV
df0 = yf.download(ticker,  period='1y') #start=start_date, end=end_date)
df1 = yf.download("^BVSP", period='1y') #start=start_date, end=end_date)

# Calculando o retorno diário
df0['Return'] = df0['Close'].pct_change()
df1['Return'] = df1['Close'].pct_change()

# Calculando a força relativa
df0['RS'] = df0['Return'].rolling(window=10).mean() - df1['Return'].rolling(window=10).mean()

# Calculando o momentum da força relativa
df0['Momentum'] = df0['RS'].diff()

# Criando DataFrame para plotagem e removendo NaN
data = pd.DataFrame({
    'Date': df0.index,
    'RS': df0['RS'],
    'Momentum': df0['Momentum']
}).dropna()

# Convertendo a coluna de datas para string
data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')

# Criando o gráfico interativo com plotly
fig = px.scatter(
    data,
    x='RS',
    y='Momentum',
    animation_frame='Date',
    range_x=[data['RS'].min(), data['RS'].max()],
    range_y=[data['Momentum'].min(), data['Momentum'].max()],
    title=f'Relative Rotation Graph (RRG) for {ticker}',
    labels={'RS': 'Relative Strength', 'Momentum': 'Momentum'},
    template='plotly_dark',
    size_max=50,  # Tamanho máximo dos pontos
    opacity=0.8,
    color_discrete_sequence=['white'],  # Cor dos pontos
)

# Adicionando linhas para indicar os quadrantes
fig.add_shape(
    type='line',
    x0=data['RS'].min(),
    x1=data['RS'].max(),
    y0=0,
    y1=0,
    line=dict(color='yellow', dash='dash'),
)
fig.add_shape(
    type='line',
    x0=0,
    x1=0,
    y0=data['Momentum'].min(),
    y1=data['Momentum'].max(),
    line=dict(color='yellow', dash='dash'),
)

# Ajustando layout para melhorar a animação e aumentar o tamanho
fig.update_layout(
    transition={'duration': 500},
    xaxis_title='Relative Strength',
    yaxis_title='Momentum',
    width=1000,  # Largura do gráfico
    height=800,  # Altura do gráfico
)

fig.show()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


- Eixo X (RS – Relative Strength): Mede a força relativa do ativo em relação ao IBOV.
    Valor positivo → ativo está superando o IBOV.
    Valor negativo → ativo está ficando atrás do IBOV.
- Eixo Y (Momentum da RS): Mede se a força relativa está acelerando ou desacelerando.
    Valor positivo → força relativa está melhorando.
    Valor negativo → força relativa está piorando.

Os Quadrantes:
 1. Quadrante Superior Direito (RS > 0, Momentum > 0)
Lideres/Fortalecendo
- O ativo está performando melhor que o IBOV.
- E está acelerando essa vantagem.
⚡️ Sinal de força e possível oportunidade de entrada.

 2. Quadrante Inferior Direito (RS > 0, Momentum < 0)
Enfraquecendo
- O ativo ainda está melhor que o IBOV, mas perdendo força.
- Momentum negativo indica que a liderança pode estar enfraquecendo.
⚠️ Atenção: pode estar no topo e prestes a inverter.

 3. Quadrante Inferior Esquerdo (RS < 0, Momentum < 0)
Perdedores / Fracos
- O ativo está ficando atrás do IBOV.
- E a fraqueza está aumentando.
🚫 Sinal de risco. Evitar ou considerar saída.

 4. Quadrante Superior Esquerdo (RS < 0, Momentum > 0)
Melhorando
- O ativo ainda está abaixo do IBOV, mas começa a ganhar força.
- Pode estar iniciando uma reversão positiva.
👀 Acompanhar: possível ponto de entrada futura.

💡 Interpretação Dinâmica (com a animação)
O diferencial do gráfico animado é que você pode visualizar a trajetória do ativo ao longo do tempo, vendo se ele:

Está migrando dos quadrantes fracos para fortes (bom sinal),

Ou indo de fortes para fracos (sinal de cautela).

